In [168]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
import sys

In [169]:
sys.path.insert(1, "../scripts")
from ml_processors import ML_Processor as MLP

mlp = MLP()

Loading data

In [193]:
ml_data = pd.read_csv('../data/chrome_data.csv')
ml2_data = pd.read_csv('../data/chrome_mobile_webView_data.csv')
ml3_data = pd.read_csv('../data/facebook_data.csv')
ml4_data = pd.read_csv('../data/platform_5.csv')
ml5_data = pd.read_csv('../data/platform_6.csv')
ml_data.head()

,auction_id,experiment,date,hour,device_make,browser,yes,no,SAID_YES
0,008aafdf-deef-4482-8fec-d98e3da054da,exposed,2020-07-04,16,Generic Smartphone,Chrome Mobile,1,0,1
1,00a1384a-5118-4d1b-925b-6cdada50318d,exposed,2020-07-06,8,Generic Smartphone,Chrome Mobile,0,1,0
2,00ebf4a8-060f-4b99-93ac-c62724399483,control,2020-07-03,15,Generic Smartphone,Chrome Mobile,0,1,0
3,013e45cf-e388-46a4-9c5b-a34303613940,exposed,2020-07-10,2,Generic Smartphone,Chrome Mobile,0,1,0
4,018af862-486e-4da1-a85b-71872120e57c,control,2020-07-03,15,Generic Smartphone,Chrome Mobile,1,0,1


Dropping unwanted columns

In [171]:
#for mobile view
columns_to_be_dropped = ['no','SAID_YES']
ml_data_clean = ml_data.drop(columns_to_be_dropped,axis=1)
ml_data_clean.head()

,auction_id,experiment,date,hour,device_make,browser,yes
0,008aafdf-deef-4482-8fec-d98e3da054da,exposed,2020-07-04,16,Generic Smartphone,Chrome Mobile,1
1,00a1384a-5118-4d1b-925b-6cdada50318d,exposed,2020-07-06,8,Generic Smartphone,Chrome Mobile,0
2,00ebf4a8-060f-4b99-93ac-c62724399483,control,2020-07-03,15,Generic Smartphone,Chrome Mobile,0
3,013e45cf-e388-46a4-9c5b-a34303613940,exposed,2020-07-10,2,Generic Smartphone,Chrome Mobile,0
4,018af862-486e-4da1-a85b-71872120e57c,control,2020-07-03,15,Generic Smartphone,Chrome Mobile,1


In [196]:
ml2_data.head(2)

,auction_id,experiment,date,hour,device_make,browser,yes,no,SAID_YES
0,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,Chrome Mobile WebView,0,1,0
1,01905bae-fc59-475e-b2b4-09b78163c961,exposed,2020-07-05,6,I3312,Chrome Mobile WebView,0,1,0


In [172]:
#for chrome mobile webview
columns_to_be_dropped = ['no','SAID_YES']
ml2_data_clean = ml2_data.drop(columns_to_be_dropped,axis=1)
ml2_data_clean.head()

,auction_id,experiment,date,hour,device_make,browser,yes
0,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,Chrome Mobile WebView,0
1,01905bae-fc59-475e-b2b4-09b78163c961,exposed,2020-07-05,6,I3312,Chrome Mobile WebView,0
2,02a9eba3-58b0-4759-b3ae-68a19cb2b59e,exposed,2020-07-09,20,Moto $2,Chrome Mobile WebView,0
3,0332792f-0de9-4902-9d08-06033c3d8371,control,2020-07-06,2,Samsung SM-J600FN,Chrome Mobile WebView,0
4,03ee1780-1fa3-4bec-926c-d543a81a6653,exposed,2020-07-04,11,Samsung SM-G981B,Chrome Mobile WebView,1


In [195]:
ml3_data.head(2)

,auction_id,experiment,date,hour,device_make,browser,yes,no,SAID_YES
0,00b6fadb-10bd-49e3-a778-290da82f7a8d,control,2020-07-08,4,Samsung SM-A202F,Facebook,1,0,1
1,023458f7-122d-475f-9525-1b9a3cc7a033,control,2020-07-03,15,MRD-LX1,Facebook,0,1,0


In [194]:
#for chrome mobile facebook
columns_to_be_dropped = ['no','SAID_YES']
ml3_data_clean = ml2_data.drop(columns_to_be_dropped,axis=1)
ml3_data_clean.head()

,auction_id,experiment,date,hour,device_make,browser,yes
0,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,Chrome Mobile WebView,0
1,01905bae-fc59-475e-b2b4-09b78163c961,exposed,2020-07-05,6,I3312,Chrome Mobile WebView,0
2,02a9eba3-58b0-4759-b3ae-68a19cb2b59e,exposed,2020-07-09,20,Moto $2,Chrome Mobile WebView,0
3,0332792f-0de9-4902-9d08-06033c3d8371,control,2020-07-06,2,Samsung SM-J600FN,Chrome Mobile WebView,0
4,03ee1780-1fa3-4bec-926c-d543a81a6653,exposed,2020-07-04,11,Samsung SM-G981B,Chrome Mobile WebView,1


In [ ]:
ml4_data.head(2)

In [ ]:
#platform_5


#   Splitting Numerical and catagoriacal vaiables

In [173]:
#for chrome mobile view
cat, num = mlp.sep_cat_num(ml_data_clean)
print("Numerical Columns:", cat)
print("****************")
print("Categorical Columns:", num)

Numerical Columns: ['auction_id', 'experiment', 'date', 'device_make', 'browser']
****************
Categorical Columns: ['hour', 'yes']


In [174]:
#for chrome mobile web view
cat, num = mlp.sep_cat_num(ml2_data_clean)
print("Numerical Columns:", cat)
print("****************")
print("Categorical Columns:", num)

Numerical Columns: ['auction_id', 'experiment', 'date', 'device_make', 'browser']
****************
Categorical Columns: ['hour', 'yes']


# Splitting the columns for encoding and label encoding

In [175]:
ml_data_clean[cat].describe()

,auction_id,experiment,date,device_make,browser
count,695,695,695,695,695
unique,695,2,8,14,1
top,008aafdf-deef-4482-8fec-d98e3da054da,exposed,2020-07-03,Generic Smartphone,Chrome Mobile
freq,1,371,176,665,695


In [176]:
ml2_data_clean[cat].describe()

,auction_id,experiment,date,device_make,browser
count,227,227,227,227,227
unique,227,2,8,79,1
top,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-03,Generic Smartphone,Chrome Mobile WebView
freq,1,180,48,43,227


In [177]:
pipe_1 = Pipeline(
    steps=[
        ("label categories", FunctionTransformer(mlp.cat_labeler, kw_args={"cat_cols": cat})),
        ("scale data", FunctionTransformer(mlp.scaler)),
        ("separate target and features", FunctionTransformer(mlp.target_feature, kw_args={"f_r": [0, 6], "t":-1})),
        ("divide dataset", FunctionTransformer(mlp.set_splitter, kw_args={"test": 0.1, "val":0.2, "rand_state":8})),      
        ])
sets = pipe_1.fit_transform(ml_data_clean)

cat_labeler output...

   auction_id  experiment  date  hour  device_make  browser  yes
0           0           1     1    16            1        0    1
1           1           1     3     8            1        0    0


scaler output... 

   auction_id  experiment      date      hour  device_make  browser  yes
0    0.000000         1.0  0.142857  0.695652     0.076923      0.0  1.0
1    0.001441         1.0  0.428571  0.347826     0.076923      0.0  0.0


target_features output... 

features size: (695, 6)


set_splitter output... 

X_train shape: (512, 6)
y_train shape: (512,)
x_test shape: (70, 6)
y_test shape: (70,)
X_val shape: (113, 6)
y_val shape: (113,)


In [178]:
pipe_2 = Pipeline(
    steps=[
        ("label categories", FunctionTransformer(mlp.cat_labeler, kw_args={"cat_cols": cat})),
        ("scale data", FunctionTransformer(mlp.scaler)),
        ("separate target and features", FunctionTransformer(mlp.target_feature, kw_args={"f_r": [0, 6], "t":-1})),
        ("divide dataset", FunctionTransformer(mlp.set_splitter, kw_args={"test": 0.1, "val":0.2, "rand_state":8})),      
        ])
sets = pipe_2.fit_transform(ml2_data_clean)

cat_labeler output...

   auction_id  experiment  date  hour  device_make  browser  yes
0           0           1     2     2            3        0    0
1           1           1     2     6           11        0    0


scaler output... 

   auction_id  experiment      date      hour  device_make  browser  yes
0    0.000000         1.0  0.285714  0.090909     0.038462      0.0  0.0
1    0.004425         1.0  0.285714  0.272727     0.141026      0.0  0.0


target_features output... 

features size: (227, 6)


set_splitter output... 

X_train shape: (167, 6)
y_train shape: (167,)
x_test shape: (23, 6)
y_test shape: (23,)
X_val shape: (37, 6)
y_val shape: (37,)


In [182]:
pipe_1 =Pipeline([('scalar3',StandardScaler()),('rf_classifier',RandomForestClassifier(random_state=2))])
pipe_2 =Pipeline([('scalar3',StandardScaler()),('rf_classifier',RandomForestClassifier(random_state=2))])

In [183]:
pipe_1.fit(sets[0], sets[1])
pipe_1.score(sets[4],sets[5])



0.5405405405405406

In [185]:
pipe_2.fit(sets[0], sets[1])
pipe_2.score(sets[4],sets[5])

0.5405405405405406